# New Attack on GeMSS

With this notebook we intent to provide the details necessary to follow our paper more closely.

## GeMSS Public Key Generaion

We start by constructing the GeMSS public key, following the notation presented in https://eprint.iacr.org/2020/1424.pdf

For simplicity in the description of the attack, we restrict the scope of this notebook to public keys formed by homogeneous quadratic polynomials.

$
\newcommand{\mat}[1]{\boldsymbol{#1}} % font for matrix
\newcommand{\ff}[1]{\mathbb{F}_{#1}} % finite field
\newcommand{\Kf}{\mathbb{K}}
\newcommand{\T}{\mathbb{T}}
\newcommand{\B}{\mathbb{B}}
\newcommand{\Fq}{\ff{q}}
\newcommand{\bigO}[1]{\ensuremath{\mathcal{O}\left(#1\right)}}
\newcommand{\ptitO}[1]{\ensuremath{o\left(#1\right)}}
\newcommand{\Fqm}{\ff{q^m}}
\newcommand{\ZZ}{\mathbb{Z}} % integers
\newcommand{\ZZp}{\mathbb{Z}_{>0}} % positive integers
\newcommand{\NN}{\mathbb{N}} % nonnegative integers
\newcommand{\cv}{\mat{c}}
\newcommand{\ev}{\mat{e}}
\newcommand{\vv}{\mat{v}}
\newcommand{\av}{\mat{a}}
\newcommand{\rv}{\mat{r}}
\newcommand{\sv}{\mat{s}}
\newcommand{\lambdav}{\mat{\lambda}}
\newcommand{\pv}{\mat{p}}
\newcommand{\uv}{\mat{u}}
\newcommand{\gv}{\mat{g}}
\newcommand{\tv}{\mat{t}}
\newcommand{\hv}{\mat{h}}
\newcommand{\xv}{\mat{x}}
\newcommand{\yv}{\mat{y}}
\newcommand{\sm}{\mat{S}} % matrix "support of e"
\newcommand{\zerom}{\mat{0}}
\newcommand{\Am}{\mat{A}}
\newcommand{\Bm}{\mat{B}}
\newcommand{\Km}{\mat{K}}
\newcommand{\Cm}{\mat{C}}
\newcommand{\Dm}{\mat{D}}
\newcommand{\Em}{\mat{E}}
\newcommand{\Fm}{\mat{F}}
\newcommand{\Zm}{\mat{Z}}
\newcommand{\Gm}{\mat{G}}
\newcommand{\Hm}{\mat{H}}
\renewcommand{\Im}{\mat{I}}
\newcommand{\Lm}{\mat{L}}
\newcommand{\Mm}{\mat{M}}
\newcommand{\Pm}{\mat{P}}
\newcommand{\Qm}{\mat{Q}}
\newcommand{\Rm}{\mat{R}}
\newcommand{\Sm}{\mat{S}}
\newcommand{\Tm}{\mat{T}}
\newcommand{\Um}{\mat{U}}
\newcommand{\Vm}{\mat{V}}
\newcommand{\Xm}{\mat{X}}
\newcommand{\Ym}{\mat{Y}}
\newcommand{\Wm}{\mat{W}}
\newcommand{\word}[1]{\vec{\boldsymbol{#1}}} % font for vector
\newcommand{\zerov}{\word{0}} % zero vector
\newcommand{\onev}{\mathds{1}} % 1 vector
\newcommand{\ident}{\mat{I}} % identity matrix
\newcommand{\trsp}[1]{#1^\mathsf{T}} % transpose matrix
$

### HFEv-

The GeMSS signature scheme is a big field cryptosystem based on the HFE construction together with the Minus and Vinegar modifiers. 

- The secret central polynomial $f:\ff{q^n}\times\ff{q}^{v}\rightarrow\ff{q^n}$ is of the form
\begin{equation}\label{eq:central}
f(X,\yv_v) = \sum_{\substack{i,j \in \mathbb{N} \\ q^i + q^j \leq D }}\alpha_{i,j}X^{q^i + q^j} +  \sum_{\substack{i \in \mathbb{N} \\ q^i \leq D }}\beta_{i}(\yv_v)X^{q^i} + \gamma(\yv_v),
\end{equation} 
where $\yv_v=(y_1,\dots,y_{v})$ are the vinegar variables, $\alpha_{i,j} \in \ff{q^n}$, the $\beta_{i}$'s are linear maps $\ff{q}^v \rightarrow \ff{q^n}$ and $\gamma$ is a quadratic map $\ff{q}^v \rightarrow \ff{q^n}$. 


- The special shape of such an $f$ gives rise to a quadratic central map over the base field $\mathcal{F} = \phi \circ f \circ \psi : \ff{q}^{n+v} \rightarrow \ff{q}^n$, where 
$$\begin{array}[t]{lrcl}
\psi : & \ff{q}^n \times \ff{q}^v & \longrightarrow & \ff{q^n} \times \ff{q}^v \\
    & (x,y) & \longmapsto & (\phi^{-1}(x),y). \end{array}$$
    

- The public key is then given by a quadratic map 
$\mathcal{P} = \mathcal{T} \circ \mathcal{F} \circ \mathcal{S}$, where $\mathcal{S}: \ff{q}^{n+v} \rightarrow \ff{q}^{n+v}$ and $\mathcal{T} : \ff{q}^n \rightarrow \ff{q}^{n-a}$ are secret affine maps of maximal rank. 

- For simplification, we assume in this notebook that $\mathcal{S}$ (resp. $\mathcal{T}$) is a linear map described by a matrix $\Sm\in \ff{q}^{(n+v) \times (n+v)}$ (resp. $\Tm \in \ff{q}^{n \times (n-a)}$), so that the components of $\mathcal{P} = (p_1,\dots,p_{n-a})$ are homogeneous polynomials in $N = n+v$ variables $\xv = (x_1,\dots,x_{n+v})$. 

- Finally, for $1 \leq i \leq n-a$ we recall that $\Pm_i \in \ff{q}^{(n+v)\times (n+v)}$ is the symmetric matrix associated to $p_i$ by $p_i(\xv) = \xv\Pm_i\trsp{\xv}$. 

In [1]:
from sage.coding.relative_finite_field_extension import *
from math import ceil
import itertools 
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing 

def findsubsets(Set, size):
    """
    Creates all the subsets of certain size of a given set
    
    Arguments:
    Set=any set, size=positive integer
    
    Returns: 
    a sequence with all subsets of 'Set' of size 'size'
    """
    return [set(x) for x in itertools.combinations(Set, size)]

def HFEv_central_frob(q, n, v, Mat):
    """
    Computes the matrix associated with the qth power of a central map in HFEv
    
    Arguments: 
    q=size of base field, n=size of the extension field,
    v=number of vinegar variables, 
    Mat=(n+v)x(n+v) matrix with the coefficients of a central map in HFEv
           
    Returns: 
    frob_F=(n+v)x(n+v) matrix with the coefficients of the qth power of the central map given by Mat
    """
    E = Mat.base_ring()
    frob_F = zero_matrix(E, n+v, n+v)
    for i,j in itertools.product(range(n), range(n)): 
        a = Mat[(i-1)%n][(j-1)%n]^q
        frob_F.add_to_entry(i,j,a)
        
    for i in range(n):
        for j in range(n,n+v):
            frob_F.add_to_entry(i,j,Mat[(i-1)%n][j]^q)
            frob_F.add_to_entry(j,i,Mat[j][(i-1)%n]^q)
    for i,j in itertools.product(range(n,n+v), range(n,n+v)): 
        frob_F.add_to_entry(i,j,Mat[i][j]^q)
    return frob_F

def HFEv(q, n, D, v, a):
    """
    Creates the public key in HFEv-
    
    Arguments: 
    q=size of base field, n=size of the extension field
    D=degree bound for the central map in HFEv-
    v=number of vinegar variables 
    a=minus parameter in HFEv-
           
    Returns: 
    P, F, E, U
    P=list of matrices representing the quadratic public polynomials
    F=the base field
    E=the extension field of degree n
    U=tilde_M^-1 * S^-1 matrix whose first row is the target 
      of the attack (linear variables u = u_{1},...,u_{n+v})   
    """
    d = ceil(log(D,q))
    F = GF(q)
    R_in = PolynomialRing(F, 'x')
    f = R_in.irreducible_element(n)
    E.<b> = F.extension(f)

    #V,from_V, to_V = E.vector_space(F, map=True)
    A = zero_matrix(E, n, n)
    B = zero_matrix(E, n, v)
    C = zero_matrix(E, v, v)
    for i,j in itertools.product(range(n), range(n)):
        if q^i + q^j <= D:
            e = E.random_element()
            if i == j:
                A.add_to_entry(i,i,e)
            else:
                A.add_to_entry(i,j,e)
                A.add_to_entry(j,i,e)
            
    for i in range(v):
        e = E.random_element()
        C.add_to_entry(i,i,e)
        for j in range(i + 1,v):         
            e = E.random_element()
            C.add_to_entry(i,j,e)
            C.add_to_entry(j,i,e)

    for i,j in itertools.product(range(n), range(v)):
        if q^i <= D:
            B.add_to_entry(i,j,E.random_element())
    central_F = block_matrix([[A,B], [B.transpose(),C]])
    frobs_F = [central_F ]
    for k in range(n-1):
        frobs_F.append(HFEv_central_frob(q,n,v,frobs_F[k])) 

    M = matrix([[b ** (j * q **i) for j in range(n)] for i in range(n)]).transpose()
    tilde_M = block_diagonal_matrix(M,identity_matrix(E,v))
    rank = 0
    while rank < n+v:
        S = random_matrix(F,n+v,n+v)
        rank = S.rank()
    rank = 0
    while rank < n-a:
        T = random_matrix(F,n,n-a)
        rank = T.rank()
    # we now follow the approach used in the proof of Prop 1 in
    # https://eprint.iacr.org/2020/1424.pdf
    U = tilde_M^-1 * S^-1
    inv_U = U^-1
    W = M^-1 * T
    LL = [inv_U*frobs_F[i]*inv_U.transpose() for i in range(n)]
    P = [sum([LL[i]*W[i][j] for i in range(n)]) for j in range(n-a)]
    return P, F, E, U

Let $\Theta := (\theta_1,\dots,\theta_n)$ be a vectorial basis of $\ff{q^n}$ over $\ff{q}$, let $\Mm \in \ff{q^n}^{n \times n}$ be the associated Moore matrix defined by 

$$\Mm := \begin{pmatrix}
\theta_1 & \theta_{1}^q &  \dots & \theta_{1}^{q^{n-1}}\\
\theta_2 & \theta_{2}^q &  \dots & \theta_{2}^{q^{n-1}}\\
\vdots & \vdots & \ddots &\vdots \\
\theta_n & \theta_{n}^q &  \dots & \theta_{n}^{q^{n-1}}
\end{pmatrix},$$
and let $\widetilde{\Mm} := \begin{pmatrix} \Mm & 0 \\ 0 & \Im_v \end{pmatrix}$. The main step of the attack is by solving the following MinRank problem to recover the first $n$ rows of $\Um := \widetilde{\Mm}^{-1}\Sm^{-1} \in \ff{q}^{(n+v) \times (n+v)}$.

### Underlying MinRank problem:

Let $d := \left\lceil \log_q{(D)} \right \rceil$, let $\uv \in \ff{q^n}^{n+v}$ be the first row of $\Um$ and let $\Pm_1, \dots, \Pm_{n-a}$ be the matrices from the public key. For $1 \leq i \leq n+v$, let $\ev_{i}$ the $i$-th canonical vector and let $\Mm_i \in \ff{q}^{(n-a) \times (n+v)}$ such that


$$\Mm_i := \ev_i\Pm_* := \begin{pmatrix} \ev_i \Pm_1 \\ \vdots \\ \ev_i \Pm_{n-a} \end{pmatrix}.$$
Then, the vector $\uv := (\uv_1,\dots,\uv_{n+v})$ is a solution to the MinRank instance described by the $\Mm_i$'s with target rank $d$. 

We set 
\begin{equation}\label{eq:matZ}
\Zm := \sum_{i=1}^{n+v} u_i \Mm_i \in \ff{q}[\uv]^{(n-a) \times (n+v)}.
\end{equation}

The matrix $\Um$ can be recovered completely from only one of the solutions to this MinRank problem. Indeed, its first $n$ rows are the Frobenius iterates of $\uv$, and the remaining $v$ rows are randomly chosen so that the final matrix is invertible. 







We choose to attack a submatrix of $\trsp{\Zm} \in \ff{q}[\uv]^{(n-a) \times (n+v)}$ by selecting a subset $J$ of $n' \leq n-a$ columns. In this notebook we choose the first $n'$ columns of $\trsp{\Zm}$.

Recall that the SM approach then consists in factoring this submatrix as
\begin{equation*}
\trsp{\Zm}_{*,J} = \Dm\Cm,
\end{equation*}
where $\Dm \in \ff{q^n}^{(n+v) \times d}$ and $\Cm \in \ff{q^n}^{d \times n'}$. The final equations are then bilinear in the linear variables $u_i$'s and in the minor variables $c_T := \left\vert \Cm \right\vert_{*,T}, T \subset \{1..n'\},~\#T = d$ which are the maximal minors of $\Cm$.


Up to relabelling of the linear variables, one can fix $u_{n+v}=1$. In this case, one expects to obtain $n$ solutions which correspond to the first $n$ rows of $\Um$, namely $\uv,\uv^{[1]},\dots,\uv^{[n-1]}$. Also, since we can choose an arbitrary submatrix $\trsp{\Zm}_{*,J}$ of $\trsp{\Zm}$ with $\# J = n'$, we can make sure that this submatrix is full rank on its first $d$ columns. Therefore, we will fix the minor variable $c_{\lbrace 1 \dots d \rbrace}$ to $1$. 


### Modeling 1 (Support-Minors modeling on $\trsp{\Zm})$

Let $\Zm$ be as defined above. We consider the SM equations obtained by choosing $n' \leq n-a$ columns in $\trsp{\Zm}$, with coefficients in $\ff{q}$ and solutions in $\ff{q^n}$. Moreover, we fix $u_{n+v}=1$ and $c_{\lbrace 1 \dots d \rbrace} = 1$. 


In [2]:
def np_opt(n, d):
    """
    Computes the optimal np so that np>=2d+1 and ncT > nu that allow to solve at degree 2
    
    Arguments: 
    n=size of the extension field
    d=the target rank = ceil(log(D,q))
           
    Returns: 
    np=number of columns of Z^T that are chosen to construct the minors 
       such that np>=2d+1 and ncT > nu

    """
    np = 2*d + 1
    ncT = binomial(np, d) - 1
    nu = n - 1
    while ncT <= nu:
        np += 1
        ncT = binomial(np, d) - 1
        
    return np
 

def HFEv_minrank_setup2(E, HFEv_pubkey, np, d): #this is for the new Attack from Jintai Ding 
    """
    Constructs the matrix Z
    
    Arguments: 
    E=extension field (big field)
    HFEv_pubkey=[P_{1},...,P_{n-a}]
               =list of matrices representing the quadratic public polynomials in HFEv-
    np=number of columns of Z^T that are chosen to construct the minors 
       at this step np is only needed to construct the right ring R
    d=the target rank = ceil(log(D,q))
           
    Returns: 
    matrix Z=[u*P_1,...,u*P_{n-a}] of order (n-a)x(n+v) with u = [u_{1},...,u_{n+v-1},1]
    R = Ring E[c_{1},...,c_{#minors-1},u_{1},...,u_{n+v-1}]
    """
    K = HFEv_pubkey[0].dimensions()[0]
    minors = findsubsets(set(range(np)), d)
    #var_names = ['c'+ str(i) for i in range(len(minors) -1)] + ['u'+ str(i) for i in range(K -1)]
    ues = ['u'+ str(i) for i in range(K -1)]
    ues.reverse()
    var_names = ['c'+ str(i) for i in range(len(minors) -1)] + ues
    R = PolynomialRing(E, names = var_names, order = "degrevlex")
    #R = PolynomialRing(E, names = ues, order = "degrevlex")
    #u = [1] + list(R.gens()[len(minors)-1:])
    # we now fix u_{n+v}=1
    u = list(R.gens()[len(minors)-1:]) + [1] 
    vector_u = vector(u)
    Z = matrix(R, len(HFEv_pubkey), K, [vector_u * HFEv_pubkey[i] for i in range(len(HFEv_pubkey))])
    return Z, R

In [3]:
def support_minor_equations2(ZT, R, np, d):
    """
    Constructs the Support-Minors equations choosing the first np columns from matrix Z^T
    
    Arguments: 
    matrix ZT=[u*P_1,...,u*P_{n-a}]^T of order (n+v)x(n-a) with u = [u_{1},...,u_{n+v-1},1]
    R = Ring E[c_{1},...,c_{#minors-1},u_{1},...,u_{n+v-1}]
    np=number of columns of ZT=Z^T that are chosen to construct the minors 
    d=the target rank = ceil(log(D,q))
        
    Returns: 
    list with all Support-Minors equations
    """
    minors = findsubsets(set(range(np)), d)
    np = min(np, ZT.dimensions()[1])
    # we now choose the mentioned submatrix of Z^T (first np columns) 
    ZTJ = ZT[:, :np]
    outputs = []
    for row in range(ZT.dimensions()[0]):
        for Set in findsubsets(set(range(np)), d+1):
            equ = 0
            Seq = list(Set)
            Seq.sort()
            for pos in range(d+1):
                aux = minors.index(Set.difference({Seq[pos]}))
                minor_var = 1
                if aux < len(minors)-1:         
                    minor_var = R.gen(aux)
                equ += minor_var * ZTJ[row][Seq[pos]] * (-1) ** pos
            outputs.append(equ)
    return outputs

### Some auxiliary functions

``cols_labels`` contains all the monomials that appear in the Support-Minors equations.

``poly_to_row(F, cols_labels, f)`` converts the polynomial ``f`` into a row of the Macaulay matrix of degree 2

In [4]:
def poly_to_row(F, cols_labels, f):
    """
    Arguments: 
    cols_labels=list with ordered monomials that represent the columns of M
    f=polynomial with monomials contained in cols_labels
             
    Returns: 
    row vector associated with f with respect to the monomials in cols_labels
    """
    #F = f.base_ring()
    Row = [F.zero() for x in range(len(cols_labels))]
    for A in zip(f.monomials(),f.coefficients()):
        Row[cols_labels.index(A[0])] = A[1]
    return vector(F, Row)

def associated_matrix(F, cols_labels, Final):
    """
    Arguments: 
    F=the base field
    cols_labels=list with ordered monomials that represent the columns of M
    Final=list of polynomials with monomials contained in cols_labels
             
    Returns: 
    matrix in which each row is associated with every polynomial in list Final
    with respect to the monomials in cols_labels
    """
    #F = Final[0].base_ring()
    return matrix(F,[poly_to_row(F, cols_labels, f) for f in Final])

def row_to_poly(R, cols_labels, row):
    """
    Arguments: 
    R = Ring E[c_{1},...,c_{#minors-1},u_{1},...,u_{n+v-1}]
    cols_labels=list with ordered monomials that represent the columns of M
    row=row vector of length #cols_labels
             
    Returns: 
    polynomial associated with the row vector row, according with the monomials in cols_labels
    """
    #R = row[0].parent()
    if row.is_zero() == True:
        return R.zero()
    else: 
        return sum([row[i]*cols_labels[i] for i in range(row.length())])
    
def associated_poly_set(F, R, cols_labels, M):
    """
    Arguments: 
    F = the base field
    R = Ring E[c_{1},...,c_{#minors-1},u_{1},...,u_{n+v-1}]
    cols_labels=list with ordered monomials that represent the columns of M
    M = matrix with #cols=#cols_labels
             
    output: 
    list with all polynomials associated with the rows of matrix M according 
    with the monomials in cols_labels
    """
    
    return [row_to_poly(R, cols_labels, M[i]) for i in range(M.dimensions()[0]) if M[i].is_zero() == False]

## Example

#### Construction of the public key

In [5]:
# GeMSS parameters
# q, n, v, D, a = (5, 16, 3, 20, 3)
# q, n, v, D, a = (5, 8, 3, 20, 2)
# q, n, v, D, a = (5, 10, 3, 20, 2)
q, n, v, D, a = (5, 12, 3, 20, 2)

# Some other parameters
d = ceil(log(D,q))
N = n + v

# Parameters for the Support-Minors Attack
# np = n-a
# np = 2*d+1
np = np_opt(n, d)
ncT = binomial(np, d) - 1
nu = n - 1

################################################
HFEv_pubkey, F, E, U = HFEv(q, n, D, v, a)

#### Step 1

In [6]:
Z, R = HFEv_minrank_setup2(E, HFEv_pubkey, np, d)
ZT = Z.transpose()
print("HFEv- Parameters:", "q = {}, D = {}, n = {}, v = {}, a = {}".format(q, D, n, v, a))
print("MinRank: n+v={} matrices of size (n+v)x(n-a)={} x {} and rank d=ceil(log(D,q))={} ".format(n+v, n+v, n-a, d))
Final = support_minor_equations2(ZT, R, np, d)
#print('b = {}'.format(b))
print("n' = {}".format(np))
################################################
Mon = []
for f in Final:
    Mon += f.monomials()
cols_labels = list(set(Mon))
cols_labels.sort()
cols_labels.reverse()
#M = associated_matrix(cols_labels, Final, R, q, m, N, d, np, b)
M = associated_matrix(F, cols_labels, Final)
print("#eqs=(n+v)*binomial(np,d+1)= {}, #monomials=(n+v)*binomial(np,d)= {}\n".format(M.dimensions()[0], M.dimensions()[1]))
print("Verification of Assumption 1: N_1 = (n+v)*binomial(np,d)-n")
Echelon_M = M.echelon_form()
Rank_M = Echelon_M.rank()
# nu = n - 1 - ((n+v)*binomial(np,d)-n - Rank_M) 
#Resultant = associated_poly_set(cols_labels, Echelon_M, R, m, N, d, np, b)
Resultant = associated_poly_set(F, R, cols_labels, Echelon_M)
#print("#minors: ", len(minors))
print("#L.I eqs = Rank of M = {}".format(Rank_M))
print("Theoretical #L.I eqs = (n+v)*binomial(np,d)-n = {}".format((n+v)*binomial(np,d)-n))
print(Rank_M == (n+v)*binomial(np,d)-n)

HFEv- Parameters: q = 5, D = 20, n = 12, v = 3, a = 2
MinRank: n+v=15 matrices of size (n+v)x(n-a)=15 x 10 and rank d=ceil(log(D,q))=2 
n' = 6
#eqs=(n+v)*binomial(np,d+1)= 300, #monomials=(n+v)*binomial(np,d)= 225

Verification of Assumption 1: N_1 = (n+v)*binomial(np,d)-n
#L.I eqs = Rank of M = 213
Theoretical #L.I eqs = (n+v)*binomial(np,d)-n = 213
True


In [7]:
print("Verification of Assumption 2: N_L = binomial(np,d)+v-1")
polys_deg1 = [f for f in Resultant if f.degree() == 1]
expected_number_polys_deg1 = binomial(np, d) - 1 + v
print("#L.I linear eqs = {}".format(len(polys_deg1)))
print("Theoretical #L.I linear eqs = binomial(np,d)-1+v = {}".format(expected_number_polys_deg1))
print(len(polys_deg1) == expected_number_polys_deg1)

Verification of Assumption 2: N_L = binomial(np,d)+v-1
#L.I linear eqs = 17
Theoretical #L.I linear eqs = binomial(np,d)-1+v = 17
True


### Modeling 2 (Quadratic System)

We consider the quadratic system in $n_u = n-1$ linear variables $u_1, \dots, u_{n-1}$ obtained __from Modeling 1__ by substitution using the linear polynomials of $\mathcal{L}$.


The first row of the matrix $U$ is formed by one instance of the linear variables $u=(u_{1},\dots,u_{n+v-1},1)$.

In [8]:
def index_minor_variable_from_linear_equation(linear_equation, num_minor_vars):
    """
    Arguments: 
    linear_equation= linear equation in E[c_{1},...,c_{#minors-1},u_{1},...,u_{n+v-1}]
    num_minor_vars= # of minor variables= binomial(np,d)-1
        
    Returns: 
    the index of the largest minor variable that appears in the linear equation 'linear_equation'
    
    Example:
    
    num_minor_vars = 15
    K = #linear variables= 14
    R = PolynomialRing(GF(5),
                            names = ['c'+ str(i) for i in range(num_minor_vars)] + 
                                    ['u' + str(num_minor_vars + i) for i in range(K)], 
                            order = "degrevlex")
    (c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14) = R.gens()[:num_minor_vars]
    (u15, u16, u17, u18, u19, u20, u21, u22, u23, u24, u25, u26, u27, u28) = R.gens()[num_minor_vars:]
    f = c7 - 2*u17 + 2*u18 - u20 - 2*u21 - 2*u23 - 2*u24 + u25 - 2*u26 + u28 + u29 - 1
    print(index_minor_variable_from_linear_equation(f, num_minor_vars) == 7)
    
    f = 2*u26 + u28 + u29 - 1
    print(index_minor_variable_from_linear_equation(f, num_minor_vars) == -1)
    """
    a = linear_equation.monomials()[0].degrees()[:num_minor_vars]
    if a.count(1) > 0:
        return a.index(1)
    else:
        # -1 when there is no minor variables
        return -1

def index_minor_variable_from_bilinear_equation(bilinear_equation, num_minor_vars):
    """
    Arguments: 
    bilinear_equation= bilinear equation in E[c_{1},...,c_{#minors-1},u_{1},...,u_{n+v-1}]
    num_minor_vars= # of minor variables= binomial(np,d)-1
        
    Returns: 
    pair (x,y), where
        x: index of the largest minor variable that appears in the bilinear equation 'bilinear_equation'
        y: index of the largest linear variable that appears in the bilinear equation 'bilinear_equation'
    
    Example:
    
    num_minor_vars = 15
    K = #linear variables = 14
    R = PolynomialRing(GF(5),
                            names = ['c'+ str(i) for i in range(num_minor_vars)] + 
                                    ['u' + str(num_minor_vars + i) for i in range(K)], 
                            order = "degrevlex")
    (c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14) = R.gens()[:15]
    (u15, u16, u17, u18, u19, u20, u21, u22, u23, u24, u25, u26, u27, u28) = R.gens()[15:]
    f = c5*u15 + u17 - u18 - 2*u20 + 2*u21 + u22 - 2*u24 - 2*u26 - u27 + 2*u28 + 1
    print(index_minor_variable_from_bilinear_equation(f, num_minor_vars)[0] == 5)
    print(index_minor_variable_from_bilinear_equation(f, num_minor_vars)[1] == 15)
    """
    a = bilinear_equation.monomials()[0].degrees()[:num_minor_vars]
    b = bilinear_equation.monomials()[0].degrees()[num_minor_vars:]
    if a.count(1) > 0:
        return a.index(1), num_minor_vars + b.index(1)
    else:
        return -1, -1
    
def get_linear_part_in_linear_variables(l, num_minor_vars):
    """
    Arguments: 
    l = c_k + a_1u_1 + ... + a_n u_n
    num_minor_vars= # of minor variables= binomial(np,d)-1
        
    Returns: 
    a_1u_1 + ... + a_n u_n for a given l = c_k + a_1u_1 + ... + a_n u_n
    """
    R = l.parent()
    return R.gen(index_minor_variable_from_linear_equation(l, num_minor_vars)) - l

### Step 1


``reduce_support_minor_equations(q, n, D, v, a, np)`` uses previous functions to produce the matrix associated with the Support-Minors modelling equations, then reduces it and outputs the polynomials associated with the rows of such matrix.

``quadratic_equations_in_linear_variables(q, n, d, np, v, Resultant, pivots)``



In [9]:
#pip install hwcounter

In [10]:
from ctypes import *
#from hwcounter import count,count_end
import hwcounter as hwc
#libc = CDLL('/usr/lib/libc.dylib')

# STEP 1
def reduce_support_minor_equations(q, n, D, v, a, np):
    """    
    Arguments: 
    q=size of base field, n=size of the extension field
    v=number of vinegar variables 
    D=degree bound for the central map in HFEv-
    a=minus parameter in HFEv-
    np=number of columns of ZT=Z^T that are chosen to construct the minors 
           
    Returns:
    matrix ZT=[u*P_1,...,u*P_{n-a}]^T of order (n+v)x(n-a) with u = [u_{1},...,u_{n+v-1},1]
    Resultant = list with all polynomials associated with the rows of the echelon form of
                matrix M according with the monomials in cols_labels
    clock_cycles_matrix_reduction = clock cycles that took to reduce matrix M
    M_rank = rank of matrix M 
    Echelon_M = Echelon form of matrix M
    pivots = List with the column indices with pivots in Echelon_M
    """
    #max_b = 15
    HFEv_pubkey, F, E, U = HFEv(q, n, D, v, a)
    d = ceil(log(D,q))
    Z, R = HFEv_minrank_setup2(E, HFEv_pubkey, np, d)
    ZT = Z.transpose()
    m = n + v
    #n_minrank = n - a
    K = n + v -1
    Set = set(range(np))
    size = d
    #minors = findsubsets(Set, size)
    #num_minor_vars = len(minors)
    # print(R)
    Final = support_minor_equations2(ZT, R, np, d)
    Final_1 = Final
    Mon = []
    for f in Final_1:
        Mon += f.monomials()
    cols_labels = list(set(Mon))
    cols_labels.sort()
    cols_labels.reverse()
    #M = associated_matrix(cols_labels, Final_1,R,q,m,K,d,np,b)
    M = associated_matrix(F, cols_labels, Final_1)
    t = hwc.count()
    #M._echelon_strassen(1024)
    #M = M.echelon_form('classical')
    Echelon_M = M.echelon_form()
    pivots = Echelon_M.pivots()
    clock_cycles_matrix_reduction = hwc.count() - t
    #Resultant = associated_poly_set(cols_labels, M, R,m,K,d,np,b)
    Resultant = associated_poly_set(F, R, cols_labels, Echelon_M)
    M_rank = Echelon_M.rank()
    #print("#L.I eqs: {}\n".format(Echelon_M.rank()))
    return ZT, Resultant, clock_cycles_matrix_reduction, M_rank, cols_labels, Echelon_M, pivots

### Step 2

In [11]:
#  STEP 2
def quadratic_equations_in_linear_variables(q, n, d, np, v, Resultant, pivots):
    """
    Arguments: 
    q=size of base field, n=size of the extension field
    v=number of vinegar variables 
    d = ceil(log(D,q))=target rank
    np = number of columns of ZT=Z^T that are chosen to construct the minors 
    Resultant = list of quadratic and linear polynomials in
               the ring E[c_{1},...,c_{#minors-1},u_{1},...,u_{n+v-1}]
               that are the output of reduce_support_minor_equations(q, n, D, v, a, np)
    pivots = List with the column indices with pivots in Echelon_M
           
    Returns: 
    polys_deg1 = the linear polynomials in Resultant
    the quadractic equations in the first n-1 linear variables u_{1},...,u_{n-1}
    """
    #u_vars_to_use_at_the_end_1 = set(range((n+v)*binomial(np,d))) - set(pivots) - {(n+v)*binomial(np,d)-1}
    #u_vars_to_use_at_the_end_1 = [var-(binomial(np, d)-1)*(n+v-1)-(binomial(np, d)-1)-v for var in u_vars_to_use_at_the_end_1]
    num_minor_vars = binomial(np, d) - 1
    output = []
    polys_deg2 = [f for f in Resultant if f.degree() == 2]
    expected_number_polys_deg2 = (n + v -1)*(binomial(np, d) - 1)
    polys_deg1 = [f for f in Resultant if f.degree() == 1]
    #print(polys_deg1)
    expected_number_polys_deg1 = binomial(np, d) - 1 + v
    R = polys_deg2[0].parent()
    # u_vars_to_use_at_the_end = the n-1 u_i's to use in the GB step
    # we start with all u_i's and then remove v of them
    u_vars_to_use_at_the_end = list(R.gens()[num_minor_vars:]) 
    u_vars_to_remove = [] #the v u_i's vars to be removed
    # cheks that we have the correct amount of bilinear and linear terms
    if len(polys_deg2) == expected_number_polys_deg2 and len(polys_deg1) == expected_number_polys_deg1: 
        given_index_f = {i:[] for i in range(num_minor_vars)}
        linear_polys_to_substitute = {}
        for l in polys_deg1:
            minor_index = index_minor_variable_from_linear_equation(l, num_minor_vars)
            # minor_index==-1 when there is no minor variable in l
            # which means l is a linear equation in the u_i's
            if minor_index == -1: 
                a = l.monomials()[0] # choose the first(largest) u_i
                u_vars_to_use_at_the_end.remove(a) # one u_i variable to be removed
                u_vars_to_remove.append(a)
            else:
                # the minor variable c_{minor_index} will be expressed in terms of linear variables?????????????
                linear_polys_to_substitute[minor_index] = get_linear_part_in_linear_variables(l, num_minor_vars)
        # We now check every quadratic polynomial
        for f in polys_deg2:
            # for f = u_i*c_j + linear_poly 
            # index_minor_variable_from_bilinear_equation(f, num_minor_vars) returns (j,i)
            bilinear_indexes = index_minor_variable_from_bilinear_equation(f, num_minor_vars) 
            if R.gen(bilinear_indexes[1]) not in u_vars_to_remove:
                given_index_f[bilinear_indexes[0]].append(f)
                l_u = linear_polys_to_substitute[bilinear_indexes[0]]
                output.append(f.substitute({R.gen(bilinear_indexes[0]): l_u}))
        Ru = PolynomialRing(GF(q), n-1, names = u_vars_to_use_at_the_end) # GF(q) or E
        #print(u_vars_to_use_at_the_end, u_vars_to_use_at_the_end_1)
        #print(u_vars_to_use_at_the_end)
        return polys_deg1, [Ru(f) for f in output]
    else:       
        print("The correct amount of bilinear terms was not found")
        
def run_one_instance(q, n, v, D, a, verbose=True):
    """
    Arguments: 
    q=size of base field 
    n=size of the extension field
    v=number of vinegar variables 
    D=degree bound for the central map in HFEv-
    a=minus parameter in HFEv-
    verbose=True or False -> determines ???????????????????????????
           
    Returns: 
    len(G) = length of the Groebner Basis 
    time_matrix_reduction = clock cycles that took to reduce the matrix associated with
                            the Support-Minors equations
    time_grobner_basis = clock cycles that took to compute the GB of the ideal generated by
                         the quadratic equations obtained after replacing in the Support-Minors equations
                         the linear equations obtained by the function 
                         reduce_support_minor_equations(q, n, D, v, a, np)
    np=number of columns of ZT=Z^T that are chosen to construct the minors, which is obtained by np_opt(n, d)
    polys_deg1 = the linear polynomials after row reducing the matrix associated with
                 the Support-Minors equations
    """
    #np, b = get_parameters_new_attack(q, n, v, D, a)
    d = ceil(log(D,q))
    np = np_opt(n, d)
    #print('np ', np)
    # STEP 1
    ZT, Resultant, time_matrix_reduction, num_li_eqs, cols_labels, Echelon_M, pivots = reduce_support_minor_equations(q, n, D, v, a, np)
    num_minor_vars = binomial(np, d) - 1
    # STEP 2
    polys_deg1, quadratic_equations = quadratic_equations_in_linear_variables(q, n, d, np, v, Resultant, pivots)
    Ru = quadratic_equations[0].parent()
    #I = ideal(Ru, quadratic_equations)
    I = Ru.ideal(quadratic_equations)
    if verbose:
        t = hwc.count()
        #G = I.groebner_basis("magma", prot="sage", deg_bound=2)
        #G = I.groebner_basis(prot=True, deg_bound=2)
        G = I.groebner_basis(prot="sage")
        #G = I.groebner_basis()
        time_grobner_basis = hwc.count() - t
        max_degree = max([poli.degree() for poli in G])
    else:
        t = hwc.count()
        #G = I.groebner_basis("magma", prot=0, deg_bound=2)
        #G = I.groebner_basis(prot=0, deg_bound=2)
        G = I.groebner_basis()
        time_grobner_basis = hwc.count() - t
    if verbose:
        print('')
        print('Maximum degree:', max_degree)
        print('')
        print('#Clock cycles Matrix reduction:', time_matrix_reduction)
        print('')
        print('#Clock cycles Grobner reduction: ', time_grobner_basis)
    #return len(G), time_matrix_reduction, time_grobner_basis, np
    return polys_deg1, G, time_matrix_reduction, time_grobner_basis, np, cols_labels, ZT

## Verification of Assumptions

### Assumption 1

Assume that $n' \geq 2d+1$. Then, the number of linearly independent equations in Modeling 1 is equal to
\begin{equation*}
    \mathcal{N}_1 := \textstyle(n+v)\binom{n'}{d} - n.
\end{equation*}

__Note:__ We choose $n'$ such that $n'\geq 2d+1$ and $n_{c_T}=\textstyle\binom{n'}{d} - 1  \geq n-1$.


### Assumption 2


By linear algebra on the affine SM equations, one can generate $\mathcal{N}_L$ linearly independent degree $1$ polynomials, where
\begin{equation}\label{eq:nb_L_def}
\mathcal{N}_L = \textstyle\binom{n'}{d} + v - 1.
\end{equation}

#### Step 2

We then consider the Macaulay matrix $\Mm(L^h)$ of the homogeneous degree $1$ parts of this system of $\mathcal{N}_L$ linear polynomials.
%The set of homogeneous parts of degree $1$ of the equations in $\mathcal{L}$ can be represented as a (Macaulay) matrix $\Lm \in \ff{q}^{\mathcal{N}_{\mathcal{L}} \times (\mathcal{N}_{\mathcal{L}} + n - 1)}$.
Here, we choose to eliminate all the $n_{c_T} := \textstyle\binom{n'}{d} - 1$ minor variables first by considering an ordering on the columns such that $c_T > u_{n+v-1} > \dots > u_2 > u_1$.

__Lema 1:__

The reduced row echelon form of $\Mm(L^h)$ is of the form 
\begin{equation}\label{eq:Ltilde}
\Lm = \begin{pmatrix}
\Im_{n_{c_T}} & * \\
0 & \Km  \\ 
\end{pmatrix} \in \ff{q}^{\mathcal{N}_L \times (n_{c_T} + n+v-1)},
\end{equation}
where $\Km \in \ff{q}^{v \times (n+v-1)}$ is row reduced.


By Lemma 1, it is possible to express all the minor variables as well as $v$ linear variables in terms of the remaining $n-1$ linear variables. Moreover, by reordering the linear variables if necessary, we may further assume that the remaining ones are $u_1,\ldots, u_{n -1}$. In this case, the matrix $\Lm$ of Lemma 1 is of the form
\begin{equation}
\Lm := \begin{pmatrix}
\Im_{n_{c_T}} & 0 & \Ym \\
0 &  \Im_v & \Wm \\ 
\end{pmatrix} \in \ff{q}^{\mathcal{N}_L \times (n_{c_T} + n+v-1)},
\end{equation}
where $~\Ym \in \ff{q}^{n_{c_T} \times n_u},~\Wm \in \ff{q}^{v \times n_u}$ and $n_u := n-1$.

### Assumption 3

The matrix $\Ym \in \ff{q}^{n_{c_T} \times n_u}$  is full rank.




In [12]:
# Use this cell to verify experimentally the 3 assumptions
# GeMSS parameters
# q, n, v, D, a = (5, 8, 3, 20, 2)
q, n, v, D, a = (5, 10, 3, 20, 2)
# q, n, v, D, a = (5, 12, 3, 20, 2)
# q, n, v, D, a = (5, 14, 3, 20, 2)
# q, n, v, D, a = (5, 16, 3, 20, 2)
# q, n, v, D, a = (5, 16, 3, 20, 3)
# q, n, v, D, a = (3, 20, 3, 5, 3)
# q, n, v, D, a = (5, 18, 3, 20, 2)

# Parameters for the Support-Minors Attack
d = ceil(log(D,q))
# np = n-a
# np = 2*d+1
np = np_opt(n, d)
# np = floor(0.5*(n-a + 2*d+1))

# Some other parameters
ncT = binomial(np, d) - 1
nu = n - 1

# expected number of LI linear polys
expected_number_polys_deg1 = binomial(np, d) - 1 + v

# number of quadratic monomials
n_quad_mon = (n + v -1)*(binomial(np, d) - 1) 

print("HFEv- Parameters:", "q = {}, D = {}, n = {}, v = {}, a = {}".format(q, D, n, v, a))
print("MinRank: n+v={} matrices of size (n+v)x(n-a)={} x {} and rank d=ceil(log(D,q))={} ".format(n+v, n+v, n-a, d))
print("n' = {}".format(np))
print("Number of eqs=(n+v)*binomial(np,d+1)= {}, Number of monomials=(n+v)*binomial(np,d)= {}"
      .format((n+v)*binomial(np,d+1), (n+v)*binomial(np,d)))
Theoretical_N_LI_eqs = (n+v)*binomial(np,d)-n 
print("Theoretical number of L.I equations = (n+v)*binomial(np,d)-n = {}".format(Theoretical_N_LI_eqs))
print("Theoretical number of L.I linear equations = N_L = binomial(np,d)+v-1 = {}\n".format(expected_number_polys_deg1))

# Choose the number to times you want to test the Assumptions
########################
Num_tries = 10 
########################
Num_coincidences = 0
suma = [[0, 0], [0, 0], [0, 0]]
      
for i in range(Num_tries):
    suma[0][0] += 1
    ZT, Resultant, t_mat_red, M_rank, cols_labels, Echelon_M, pivots = reduce_support_minor_equations(q, n, D, v, a, np)
    suma[0][1] += M_rank == Theoretical_N_LI_eqs
    if M_rank == Theoretical_N_LI_eqs:
        suma[1][0] += 1
        polys_deg1, new_polys_deg2 = quadratic_equations_in_linear_variables(q, n, d, np, v, Resultant, pivots)
        suma[1][1] += len(polys_deg1) == expected_number_polys_deg1
        
        suma[2][0] += 1
        u_vars_to_use_at_the_end = set(range((n+v)*binomial(np,d))) - set(pivots) - {(n+v)*binomial(np,d)-1}
    
        # Let us extract the matrix L from the matrix Echelon_M
        L = Echelon_M[n_quad_mon : M_rank, n_quad_mon : -1]
        # Let us extract the matrix Y from the matrix Echelon_M using the pivots of the linear part
        Y = Echelon_M.matrix_from_rows_and_columns(range(n_quad_mon, n_quad_mon+ncT),list(u_vars_to_use_at_the_end))
        suma[2][1] += Y.rank() == Y.dimensions()[1]

        
percentages = [ceil(suma[i][1]/suma[i][0]) for i in range(3)] 
print("Number of coincidences for 'Assumption 1' is {} out of {} tries".format(suma[0][1], suma[0][0]))
print("Number of coincidences for 'Assumption 2' is {} out of {} tries".format(suma[1][1], suma[1][0]))
print("Number of coincidences for 'Assumption 3' is {} out of {} tries".format(suma[2][1], suma[2][0]))
print("Number of Colums in matrix Y = {}".format(Y.dimensions()[1]))


HFEv- Parameters: q = 5, D = 20, n = 10, v = 3, a = 2
MinRank: n+v=13 matrices of size (n+v)x(n-a)=13 x 8 and rank d=ceil(log(D,q))=2 
n' = 6
Number of eqs=(n+v)*binomial(np,d+1)= 260, Number of monomials=(n+v)*binomial(np,d)= 195
Theoretical number of L.I equations = (n+v)*binomial(np,d)-n = 185
Theoretical number of L.I linear equations = N_L = binomial(np,d)+v-1 = 17

Number of coincidences for 'Assumption 1' is 10 out of 10 tries
Number of coincidences for 'Assumption 2' is 10 out of 10 tries
Number of coincidences for 'Assumption 3' is 10 out of 10 tries
Number of Colums in matrix Y = 9


## Attacking an instance of GeMSS

Choose values for the GeMSS parameters and perform the attack:

In [13]:
# GeMSS parameters
# q, n, v, D, a = (5, 8, 3, 20, 2)
# q, n, v, D, a = (5, 10, 3, 20, 2)
q, n, v, D, a = (5, 12, 3, 20, 2)
# q, n, v, D, a = (5, 14, 3, 20, 2)
# q, n, v, D, a = (5, 16, 3, 20, 2)
# q, n, v, D, a = (5, 16, 3, 20, 3)
# q, n, v, D, a = (3, 20, 3, 5, 3)
# q, n, v, D, a = (5, 18, 3, 20, 2)

polys_deg1, GB, time_reduction, time_GB, np, cols_labels, ZT = run_one_instance(q=q, n=n, v=v, D=D, a=a, verbose=True)



Maximum degree: 2

#Clock cycles Matrix reduction: 10405310

#Clock cycles Grobner reduction:  33475498


#### Verifying the solution

Next, we want to use the Groebner basis (GB) obtained by our attack to compute one solution of the MinRank problem, and then show that it indeed produces a matrix with rank $d$.

In [14]:
R = polys_deg1[0].parent()
d = ceil(log(D,q))
ncT = binomial(np, d) - 1
E = R.base_ring()
eqs3 = GB
ues = ['u'+ str(i) for i in range(n-1)]
ues.reverse()
NR = PolynomialRing(E, names = ues, order = "degrevlex")
eqs3 = [NR(ecuacion) for ecuacion in eqs3]
Im4=NR.ideal(eqs3)
V4=Im4.variety()
sub_dic3 = {R.gen(ncT+v+i): V4[0]['u'+str(n-1-i-1)] for i in range(0, n-1)}
for i in range(v):
    sub_dic3[R.gen(ncT+i)] = E((R.gen(ncT+i)-polys_deg1[ncT+i]).substitute(sub_dic3))

In [16]:
# sub_dic3 is a dictionary with a solution of the MinRank problem
print(sub_dic3)

{u10: 3*b^10 + b^9 + 4*b^8 + 4*b^6 + 2*b^5 + 2*b^4 + 3*b^3 + b^2 + b + 1, u9: 3*b^10 + 2*b^9 + b^8 + 3*b^7 + 3*b^6 + 4*b^5 + 2*b^4 + 4*b^3 + 2*b + 2, u8: b^9 + b^8 + 4*b^7 + b^6 + 2*b^5 + 2*b^4 + b^2 + 3, u7: 4*b^11 + 4*b^10 + b^9 + 3*b^8 + 3*b^7 + b^5 + 2*b^4 + 2*b^3 + 3*b^2 + 4*b + 1, u6: 2*b^10 + 4*b^9 + 3*b^8 + 4*b^7 + 3*b^5 + b^4 + b^3 + 4*b^2 + 1, u5: b^11 + b^10 + 3*b^9 + b^8 + 2*b^7 + 3*b^5 + 4*b^4 + b^3 + 2*b^2 + 4, u4: b^11 + 3*b^10 + 4*b^9 + b^8 + 4*b^7 + 4*b^6 + 2*b^5 + b^4 + 3*b^3 + 1, u3: b^11 + 4*b^10 + b^9 + 3*b^8 + 4*b^7 + 4*b^6 + 4*b^5 + b^4 + 3*b^3 + 3*b, u2: b^9 + 2*b^8 + 3*b^7 + 4*b^5 + 3*b^4 + b^2 + b, u1: 3*b^11 + 4*b^9 + 3*b^8 + 4*b^7 + 3*b^6 + 3*b^5 + 3*b^4 + 3*b^2 + 3*b + 3, u0: b^10 + 2*b^9 + 3*b^6 + 3*b^5 + 3*b^4 + 2*b^3 + 2*b^2 + 3, u13: 4*b^10 + 3*b^9 + 3*b^8 + 3*b^6 + 2*b^4 + b^3 + 2*b^2 + 4, u12: b^11 + b^10 + 2*b^9 + 4*b^8 + 4*b^6 + b^5 + b^3 + 2*b^2 + 3, u11: 3*b^11 + 3*b^10 + 3*b^9 + 4*b^8 + 2*b^7 + b^6 + 2*b^4 + 3*b^3 + 2*b^2}


In [17]:
# Here we check that the rank of Z^{T} is in fact d
RZT=matrix([[E(ZT[i, j].substitute(sub_dic3)) for j in range(ZT.dimensions()[1])] 
            for i in range(ZT.dimensions()[0])]).rank() 

print("Rank of Z^{T}=d:", RZT == d)

Rank of Z^{T}=d: True


## Time complexity of the attack

In [18]:
#(ngemss, v, D, a)
parameters = {
  'GeMSS128': (174, 12, 513, 12),
  'BlueGeMSS128': (175, 14, 129, 13),
  'RedGeMSS128':(177,15,17,15),
  'GeMSS192': (265, 20, 513, 22),
  'BlueGeMSS192':(265, 23, 129, 22),
  'RedGeMSS192': (266, 25, 17, 23),
  'GeMSS256': (354, 33, 513, 30),
  'BlueGeMSS256': (358, 32, 129, 34),
  'RedGeMSS256': (358, 35, 17, 34),
}

print("Time Complexity of our Attack on GeMSS (log_2(#gates))\n")
print(f'Scheme \t\t (q, n, v, D, a) \t\t Step_1_S \t Step_1_BW \t Step_2 \t np')
print('')
q = 2
omega = 2.81

for scheme, param in parameters.items():
    n, v, D, a = param
    d = ceil(log(D, 2))
    #w = (d + 1) * (n + v)
    #np = 2*d+1
    np = np_opt(n, d)
    nu = n-1
    ncT = binomial(np, d)-1

    # STEP 1
    #complex_1_S = ceil(log(ncT^omega*nu^omega, 2))
    complex_1_S = ceil(log((ncT+1)^omega*(n+v)^omega, 2))
    #complex_1_W = ceil(log(d*ncT^2*nu^4, 2))
    complex_1_W = ceil(log(n*(n+v)^3*(d+1)*(ncT+1)^2, 2))
    
    # STEP 2
    complex_2 = ceil(log(ncT*nu^(2*omega-1), 2))
    
    print('{} \t ({},{},{},{},{})  \t\t {} \t\t {} \t\t {} \t\t {}'
          .format(scheme, q, n, v, D, a, complex_1_S, complex_1_W, complex_2, np))

Time Complexity of our Attack on GeMSS (log_2(#gates))

Scheme 		 (q, n, v, D, a) 		 Step_1_S 	 Step_1_BW 	 Step_2 	 np

GeMSS128 	 (2,174,12,513,12)  		 73 		 71 		 53 		 21
BlueGeMSS128 	 (2,175,14,129,13)  		 63 		 63 		 49 		 17
RedGeMSS128 	 (2,177,15,17,15)  		 47 		 51 		 44 		 11
GeMSS192 	 (2,265,20,513,22)  		 75 		 73 		 56 		 21
BlueGeMSS192 	 (2,265,23,129,22)  		 64 		 65 		 52 		 17
RedGeMSS192 	 (2,266,25,17,23)  		 48 		 53 		 47 		 11
GeMSS256 	 (2,354,33,513,30)  		 76 		 75 		 58 		 21
BlueGeMSS256 	 (2,358,32,129,34)  		 66 		 67 		 54 		 17
RedGeMSS256 	 (2,358,35,17,34)  		 50 		 55 		 49 		 11


#### With Projection

In [19]:
#(ngemss, v, D, a, p)
parameters = {
  'GeMSS128': (174, 12, 513, 12, 0),
  'BlueGeMSS128': (175, 14, 129, 13, 1),
  'RedGeMSS128':(177,15,17,15, 4),
  'GeMSS192': (265, 20, 513, 22, 5),
  'BlueGeMSS192':(265, 23, 129, 22, 7),
  'RedGeMSS192': (266, 25, 17, 23, 10),
  'GeMSS256': (354, 33, 513, 30, 10),
  'BlueGeMSS256': (358, 32, 129, 34, 11),
  'RedGeMSS256': (358, 35, 17, 34, 14),
}

print("Time Complexity of our Attack on GeMSS with projection (log_2(#gates))\n")
print(f'Scheme \t\t (q, n, v, D, a) \t p \t\t Step_1_S \t Step_1_BW \t Step_2 \t np')
print('')
q = 2
omega = 2.81

for scheme, param in parameters.items():
    n, v, D, a, p = param
    d = ceil(log(D, 2))
    d = d + p
    #w = (d + 1) * (n + v)
    #np = 2*d+1
    np = np_opt(n, d)
    nu = n-1
    ncT = binomial(np, d)-1

    # STEP 1
    #complex_1_S = ceil(log(ncT^omega*nu^omega, 2))
    complex_1_S = ceil(log((ncT+1)^omega*(n+v)^omega, 2))
    #complex_1_W = ceil(log(d*ncT^2*nu^4, 2))
    complex_1_W = ceil(log(n*(n+v)^3*(d+1)*(ncT+1)^2, 2))
    
    # STEP 2
    complex_2 = ceil(log(ncT*nu^(2*omega-1), 2))
    
    print('{} \t ({},{},{},{},{})  \t {} \t\t {} \t\t {} \t\t {} \t\t {}'
          .format(scheme, q, n, v, D, a, p, complex_1_S, complex_1_W, complex_2, np))

Time Complexity of our Attack on GeMSS with projection (log_2(#gates))

Scheme 		 (q, n, v, D, a) 	 p 		 Step_1_S 	 Step_1_BW 	 Step_2 	 np

GeMSS128 	 (2,174,12,513,12)  	 0 		 73 		 71 		 53 		 21
BlueGeMSS128 	 (2,175,14,129,13)  	 1 		 68 		 67 		 51 		 19
RedGeMSS128 	 (2,177,15,17,15)  	 4 		 68 		 67 		 51 		 19
GeMSS192 	 (2,265,20,513,22)  	 5 		 103 		 93 		 66 		 31
BlueGeMSS192 	 (2,265,23,129,22)  	 7 		 103 		 93 		 66 		 31
RedGeMSS192 	 (2,266,25,17,23)  	 10 		 103 		 93 		 66 		 31
GeMSS256 	 (2,354,33,513,30)  	 10 		 131 		 115 		 78 		 41
BlueGeMSS256 	 (2,358,32,129,34)  	 11 		 126 		 111 		 76 		 39
RedGeMSS256 	 (2,358,35,17,34)  	 14 		 126 		 111 		 76 		 39


## Space complexity of the attack


In [20]:
def memory_bw(q, w, cols): 
    return w * log(q, 2) * cols * log(cols, 2) 

#(ngemss, v, D, a)
parameters = {
  'GeMSS128': (174, 12, 513, 12),
  'BlueGeMSS128': (175, 14, 129, 13),
  'RedGeMSS128':(177,15,17,15),
  'GeMSS192': (265, 20, 513, 22),
  'BlueGeMSS192':(265, 23, 129, 22),
  'RedGeMSS192': (266, 25, 17, 23),
  'GeMSS256': (354, 33, 513, 30),
  'BlueGeMSS256': (358, 32, 129, 34),
  'RedGeMSS256': (358, 35, 17, 34),
}

In [21]:

print('Space complexity log2(# bytes)')
print('')
print(f'Scheme \t\t normal_BW \t optimized_BW')
print('')
for scheme, param in parameters.items():
    n, v, D, a = param
    d = ceil(log(D, 2))
    w = (d + 1) * (n + v)
    n_prime = 2 * d + 1
    num_minors_of_Z = binomial(n_prime, d + 1)
    #V_1: Space to store initial matrices 
    space_initial_matrices = (n + v) * n_prime * (n + v)
    #V_2: Space to store indexes of the Macaulay Matrix
    space_index_vector_macaulay = num_minors_of_Z * log(num_minors_of_Z * (n + v), 2) * (d + 1) * (n + v)
    #V_3: Space to store indexes of the of the coefficient matrix
    space_index_vector_coefficient = num_minors_of_Z * log(n_prime * (n + v)) * (d + 1) * (n + v)
    #V_4: Space to store index row in the coefficient matrix associated in a given row of the Macaulay matrix
    space_index_to_row_coefficient = binomial(n_prime, d) * (n + v) * log(n + v, 2)
    #Total space by BW
    temp1 = space_initial_matrices
    temp2 = space_index_vector_macaulay
    temp3 = space_index_vector_coefficient
    temp4 = space_index_to_row_coefficient
    total_space_optimized_BW = temp1 + temp2 +  temp3 + temp4
                    
    #Complexity normal BW
    total_space_normal_BW = binomial(n_prime, d)* (n + v) * (n + v) * (d + 1) * log(binomial(n_prime, d) * (n + v), 2)
    print('{} \t {:.3f} \t {:.3f}'.format(scheme, float(log(total_space_normal_BW / 8, 2)), float(log(total_space_optimized_BW, 2))))

Space complexity log2(# bytes)

Scheme 		 normal_BW 	 optimized_BW

GeMSS128 	 38.665 	 34.553
BlueGeMSS128 	 34.332 	 30.258
RedGeMSS128 	 27.645 	 23.729
GeMSS192 	 39.930 	 35.213
BlueGeMSS192 	 35.586 	 30.917
RedGeMSS192 	 28.897 	 24.410
GeMSS256 	 40.836 	 35.686
BlueGeMSS256 	 36.488 	 31.389
RedGeMSS256 	 29.800 	 24.905


In [22]:
for scheme, param in parameters.items():
    n, v, D, a = param
    d = ceil(log(D, 2))
    w = (d + 1) * (n + v)
    n_prime = 2 * d + 1
    print('{} \t {:.3f} '.format(scheme,float(log(binomial(n_prime, d)**2 * (n + v)**2 /  8, 2))))

GeMSS128 	 48.935 
BlueGeMSS128 	 41.263 
RedGeMSS128 	 29.873 
GeMSS192 	 50.166 
BlueGeMSS192 	 42.478 
RedGeMSS192 	 31.073 
GeMSS256 	 51.049 
BlueGeMSS256 	 43.353 
RedGeMSS256 	 31.940 


In [23]:
2**9

512

In [24]:
for scheme, param in parameters.items():
    n, v, D, a = param
    d = ceil(log(D, 2))
    w = (d + 1) * (n + v)
    n_prime = 2 * d + 1
    print('{} \t {:.3f} '.format(scheme,float(log(binomial(n_prime, d) * (n + v), 2))))

GeMSS128 	 25.967 
BlueGeMSS128 	 22.132 
RedGeMSS128 	 16.437 
GeMSS192 	 26.583 
BlueGeMSS192 	 22.739 
RedGeMSS192 	 17.037 
GeMSS256 	 27.024 
BlueGeMSS256 	 23.177 
RedGeMSS256 	 17.470 
